# Pet Breed Classification with FastAI

This notebook implements a pet breed classifier using FastAI and a pre-trained ResNet34 model. We'll go through:

1. Data preparation and loading
2. Model setup and training
3. Performance evaluation and visualization

In [4]:
# Import required libraries
from fastai.vision.all import *
from pathlib import Path
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'fastai'

## 1. Data Preparation

First, we'll set up our data processing pipeline. The images should be organized in the `data` directory with filenames starting with their category names.

In [ ]:
# Define category extraction function
def get_category(fname):
    return fname.split('_')[0]  # Extract category name before underscore

# Set data path
path = Path('data')

# Verify data directory exists
assert path.exists(), f"Data directory {path} not found!"

In [ ]:
# Create DataBlock with data augmentation
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2),  # 20% validation set
    get_y=lambda x: get_category(x.name),
    item_tfms=[Resize(224)],  # Resize images
    batch_tfms=[
        # Data augmentation transforms
        aug_transforms(
            mult=1.0,
            do_flip=True,
            flip_vert=False,
            max_rotate=10.0,
            min_zoom=1.0,
            max_zoom=1.1,
            max_lighting=0.2,
            max_warp=0.2,
            p_affine=0.75,
            p_lighting=0.75,
        ),
        Normalize.from_stats(*imagenet_stats)  # Normalize using ImageNet stats
    ]
)

# Create DataLoaders
dls = dblock.dataloaders(path, bs=32)

# Show a batch of images
dls.show_batch(max_n=9, figsize=(10,10))

## 2. Model Training

We'll use a pre-trained ResNet34 model and fine-tune it for our specific task.

In [ ]:
# Create learner
learn = vision_learner(dls, resnet34, metrics=error_rate)

# Fine-tune the model
learn.fine_tune(
    epochs=10,
    base_lr=3e-3,
    freeze_epochs=3,  # Train with frozen layers first
    cbs=[
        ShowGraphCallback(),  # Show training progress
        SaveModelCallback(monitor='valid_loss'),  # Save best model
        ReduceLROnPlateau(monitor='valid_loss', patience=2)  # Adaptive learning rate
    ]
)

## 3. Model Evaluation

Let's analyze the model's performance using various visualization techniques.

In [ ]:
# Create interpretation
interp = ClassificationInterpretation.from_learner(learn)

# Plot confusion matrix
plt.figure(figsize=(12, 12))
interp.plot_confusion_matrix()
plt.title('Confusion Matrix')
plt.show()

# Show top losses
interp.plot_top_losses(9, figsize=(15,11))
plt.show()

In [ ]:
# Save the trained model
learn.export('pet_classifier.pkl')

## 4. Model Testing

Test the model on some sample images to see how it performs.

In [ ]:
# Function to predict on a single image
def predict_image(img_path):
    img = PILImage.create(img_path)
    pred, pred_idx, probs = learn.predict(img)
    return f'Prediction: {pred}
Probability: {probs[pred_idx]:.4f}'

# Test on a sample image (uncomment and modify path as needed)
# img_path = 'path/to/test/image.jpg'
# print(predict_image(img_path))